# IBESI

**IBESI** is an all-encompassing money management system for Oscar and Rose. It stands for:
- **I**ncome
- **B**ills
- **E**xpenses
- **S**avings
- **I**nvestments

These are the key financial areas this system supports.

## Financial Areas
### Income
Income refers to any source of money coming to us. The most obvious instance of income is our salaries.

### Bills
Bills are the payments performed in cycles. These include mortgage, utility bills and similar.

### Expenses
Expenses are any ad-hoc payments for goods or services. These include food, cleaning and shopping.

### Savings
This is the share of money left after using our income to pay for bills and expenses that is saved.

### Investments
This is the share of money left after savings that is put back in the market to generate a return.

## Money Pipeline
The money processing pipeline is as shown below.

<!-- ![pipeline](ibesi-pipeline.png) -->


## Components
The components are the steps of the money pipeline. These operations are part of the core logic of the system and should not be modified without a system design review.

## Input Data
- Income amount
- Savings accumulated
- Bills for the past month
- Expenses for the past month

## Parameters
Parameters are the parts of the system we can modify to make it work for our financial situation and expectations. These include:
- Saving pots and shares for each of them
- How bills and expenses are split between us

In [3]:
from data.D2312 import INPUTS, PARAMS 

def inspect_inputs_dict():
    pass

def inspect_params_dict():
    pass

def get_current_savings(inputs):
    return sum(sum(source.values()) for source in inputs["savings"].values())

def get_bills_total(inputs):
    return (sum(sum(source.values()) for source in inputs["bills"].values()))

def get_expenses_total(inputs):
    return (sum(sum(source.values()) for source in inputs["expenses"].values()))

def get_spending_total(inputs):
    return get_bills_total(inputs) + get_expenses_total(inputs)

def get_split_percentage(inputs, params):
    if params["split"] == "proportional_to_income":
        income_oscar = sum(source["oscar"] for source in inputs["income"].values())
        income_rose = sum(source["rose"] for source in inputs["income"].values())
        split_percentage_oscar = income_oscar / (income_oscar + income_rose)
        split_percentage_rose = income_rose / (income_oscar + income_rose)
        return split_percentage_oscar, split_percentage_rose
    else:
        raise ValueError("split mode not supported")

def get_split_difference():
    pass

print(get_current_savings(INPUTS))
print(get_split_percentage(INPUTS, PARAMS))
print(get_bills_total(INPUTS))
print(get_expenses_total(INPUTS))
print(get_bills_total(INPUTS) + get_expenses_total(INPUTS))

33600
(0.6584251545283526, 0.3415748454716474)
2641.95
715.53
3357.4799999999996
